# Graphviz

We include code for converting composite bloqs to graphs, drawn with graphviz.

In [ ]:
from qualtran.drawing import GraphDrawer, PrettyGraphDrawer
from qualtran.drawing.graphviz_test import Atom, TestParallelBloq

from IPython.display import SVG

def show_bloq(bloq, draw_cls=PrettyGraphDrawer):
    display(SVG(draw_cls(bloq).get_graph().create_svg()))

Each constituent bloq is a table whose header contains the bloq name. Each row is a register. Edges represent connections between soquets. Each edge is labeled with the bitsize of the connected registers. Dangling soquets (corresponding to the enclosing bloq's registers) are drawn as plain text.

In [ ]:
show_bloq(Atom())

In [ ]:
show_bloq(TestParallelBloq())

In [ ]:
show_bloq(TestParallelBloq().decompose_bloq())

### `GraphDrawer`

The `PrettyGraphDrawer` abbreviates names, hides some details of reshaping bloqs, and chooses drawing parameters to give a compact visualization of the bloq. `GraphDrawer` does not add any styling information and relies on graphviz defaults.

In [ ]:
show_bloq(TestParallelBloq().decompose_bloq(), draw_cls=GraphDrawer)

### `pydot`

We use `pydot` to manupulate graphviz graphs. You can get that graph directly:

In [ ]:
cbloq = TestParallelBloq().decompose_bloq()
pgd = PrettyGraphDrawer(cbloq)
graph = pgd.get_graph()
print(graph.to_string())

In [ ]:
import pydot

In [ ]:
graph2, = pydot.graph_from_dot_data(graph.create_dot().decode())

In [ ]:
nodes = graph2.get_nodes()
node = nodes[2]
node.obj_dict

In [ ]:
[(node.get_name(), node.obj_dict['type']) for node in nodes]

In [ ]:
pos_dict = {node.get_name(): node.get_pos() for node in nodes}
pos_dict

In [ ]:
binst_pos = {}
for binst in cbloq.bloq_instances:
    graphviz_id = pgd.ids[binst]
    strpos = pos_dict[graphviz_id]
    if strpos is None:
        raise ValueError(f"Graphviz didn't provide a pos for {binst}")
    
    binst_pos[binst.i] = strpos

In [ ]:
binst_pos

In [ ]:
pgd.ids

In [ ]:
print(graph.create_dot().decode())

## Register Groups

We group left and right registers with shared names. This section shows some usual and unusual register specifications to test the graphviz logic for making the table rows match up.

In [ ]:
from qualtran import Bloq, Register, Signature, Side

class SignatureBloq(Bloq):
    """Placeholder bloq that lets you specify its signature."""
    def __init__(self, signature: Signature):
        self._signature = signature
        
    @property
    def signature(self) -> Signature:
        return self._signature
        
    def short_name(self):
        return 'Bloq'

In [ ]:
bloq = SignatureBloq(Signature([
    Register('x', 100),
    Register('y', 200),
]))
show_bloq(bloq)

In [ ]:
bloq = SignatureBloq(Signature([
    Register('x', 100),
    Register('a', 2, shape=(2,), side=Side.LEFT),
    Register('y', 200),
]))
show_bloq(bloq)

In [ ]:
bloq = SignatureBloq(Signature([
    Register('x', 100),
    Register('a', 2, shape=(2,2), side=Side.LEFT),
    Register('a', 8,  side=Side.RIGHT),
    Register('y', 200),
]))
show_bloq(bloq)

In [ ]:
bloq = SignatureBloq(Signature([
    Register('x', 100),
    Register('a', 2, shape=(2,2), side=Side.LEFT),
    Register('a', 4, shape=(2,), side=Side.RIGHT),
    Register('y', 200),
]))
show_bloq(bloq)